# Welcome to the accelerate python code tutorial

## This notebook has a pdf

This notebook also comes with its own pdf. You can use this pdf as a last-resort tool, when python simply won't work. The very bottom of this notebook contains some instructions for converting this notebook to pdf.

## Setting up the environment

We need some packages that do not come with standard python deployments. Either install these packages via this conda / pip commands yourself, or install the environment based on the .yaml file in this directory.

## Setting autoreload

Importing some pre-compiled cython files is similar to importing regular python files. They get appended to the current namespace and stay there even if you change the file or reload it. Let us use the autreload magic to fix that issue.

In [1]:
%load_ext autoreload
%autoreload 2

# Cython. It's compiled python.

Cython is aimed to be a superset to the python programming language. Cython is a compiled language with python-like syntax. You can write cython files (.pyx) files in a similar manner than you would write normal python files (.py), compile them and import the generated shared object (.so) as a module into your python code. Doing this allows you to greatly decrease the computational overhead at run time, i.e. make python faster. You may find the best use in cython if you use it to compile specific computationally-heavy functions.

## The compilation pipeline

Cython is always compiled in two steps.

- The cython compiler transforms cython code (.pyx) into optimized and platform-independent C or C++ code (.c).
- A standard compiler compiles the C code (.c) into a shared object (.so) which can be imported to python.

The shared object is platform specific (architecture and OS). Someone running the very niche Windows 10 for ARM will face some difficulties using your .so file. On windows these files also get a different extension (.pyd). After we get this sorted out we will look at how to write and compile cython.

## Compilation

There are several ways you can utilize to get your cython code compiled and imported into python. Here are 3:

### Distutils and Cythonize

`distutils` is a python standard package used for building, packing and distributing python projects. It can compile C source code into an extension module. How neat is that?! It manages all python versions, platforms and architectures. That's pretty neat. Cython's `cythonize()` function produces a .c file and `distutils` compiles it for us. This compilation pipeline requires us to write a short setup script. This compilation pipeline is the most common way to ditribute cython code.



**Reading and writing files in jupyter**

Instead of opening with your text editor of choice (Hint: There is a correct answer to the question "What is your favorite text editor" and emacs is not it.) let us open the .pyx files direclty in this notebook using magic. With magic I am talking about the stuff you put after the percent characters in cells. This includes line magic `%matplotlib notebook` and  cell magic `%%time`.

In [1]:
%%time
for i in range(9):
    y = i ** i ** i

CPU times: user 372 ms, sys: 16 ms, total: 388 ms
Wall time: 385 ms


Let's use the %load magic function to open the file helloworld.pyx

In [14]:
# %load helloworld.pyx
print(Hello World)

As you can probably guess, the quotation marks declaring 'Hello World' as a string are missing. Let's fix that using the %%writefile cell magic.

In [2]:
%%writefile helloworld.pyx
def hello_world():
    """This is the docstring which gets transferred into the .so
    
    Additionally to printing hello world, I will also do some calculations.
    
    Args:
        No args lol.
        
    Returns:
        None
        
    """
    for i in range(9):
        y = i ** i ** i
    print("Hello World")

Overwriting helloworld.pyx


**The setup file**

A simple setup.py file is only a few lines long (most of them are imports). At the core you have the nested function call. The `cythonize()` function can take single files, list of files or glob patterns. The `cythonize()` function returns a list of distutils extension modules, that the setup function knows how to use. This setup.py file can be invoked from the command-line using:

```bash
$ python setup.py build_ext --inplace
```

In [2]:
%%writefile setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("helloworld.pyx")
)

Overwriting setup.py


In [3]:
!python setup.py build_ext --inplace

running build_ext


In [4]:
import os
os.listdir()

['accelerate_python_code.pdf',
 'build',
 'helloworld.c',
 'helloworld.cpython-36m-x86_64-linux-gnu.so',
 'helloworld.pyx',
 'prime_numbers.c',
 'prime_numbers.cpython-36m-x86_64-linux-gnu.so',
 'prime_numbers.pyx',
 'prime_numbers_cdef.c',
 'prime_numbers_cdef.cpython-36m-x86_64-linux-gnu.so',
 'prime_numbers_cdef.pyx',
 'setup.py',
 'vmdscene.mtl',
 'vmdscene.obj',
 '.git',
 '.gitignore',
 'README.md',
 'cython_workshop.ipynb',
 '.ipynb_checkpoints']

You see, that some files have appeared in your current directory. The helloworld.c file contains quite a few lines C code. But after compiling it, it is faster than python.

In [6]:
count = len(open('helloworld.c').readlines())
print(count)

2752


Now we can import stuff from our shared object.

In [8]:
from helloworld import hello_world
print(help(hello_world))

Help on built-in function hello_world in module helloworld:

hello_world(...)
    This is the docstring which gets transferred into the .so
    
    Additionally to printing hello world, I will also do some calculations.
    
    Args:
        No args lol.
        
    Returns:
        None

None


In [9]:
hello_world()

Hello World


## IPython %%cython magic

Although distutils is the quasi-standard for deploying cython code we can use ipythons %%cyhton magic to interactively work with cython. IPython handles the rest for us, translating, compiling and importing the function. This also circumvents a major draw-back when you develop cython code.

**:( :( :( :( :( :( :( :(**

Changes in .so files are not caught by the %autoreload extension, meaning if you change something in your cython code and recompile it, you have to restart the kernel.

In [11]:
%load_ext Cython

The %%cython cell magic comes with its own options, you can check out here:

https://ipython.org/ipython-doc/2/config/extensions/cythonmagic.html

In [12]:
%%cython
def fibonacci(int n):
    cdef int i
    cdef double a=0.0, b=1.0
    for i in range(n):
        a, b = a+b, a
    return a

In [14]:
fibonacci(20)

6765.0

The compiled file can be found in ~/.ipython/cython)

## On-the-fly compilation with pyximport

pyximport retrofits the import statement to recognize .pyx extension modules. The pyximport module comes with cython and can easily be accessed via:

In [42]:
import pyximport
pyximport.install()

(None, None)

In [45]:
%%writefile fib.pyx
def fibonacci(int n):
    cdef int i
    cdef double a=0.0, b=1.0
    for i in range(n):
        a, b = a+b, a
    return a

Writing fib.pyx


With pyximport we can now import from .pyx files. In the background there is still the compilation pipeline going on, but we don't need to worry about that.

In [46]:
import fib

/home/kevin/.conda/envs/work_3/lib/python3.6/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/kevin/projects/cython_workshop/fib.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [50]:
print(fib.__file__)
print(type(fib))

/home/kevin/.pyxbld/lib.linux-x86_64-3.6/fib.cpython-36m-x86_64-linux-gnu.so
<class 'module'>


In [51]:
%%time
fib.fibonacci(90)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 12.4 µs


2.880067194370816e+18

## pyximport and further C dependencies

The great advantage of pyximport is that it not only checks, whether the source .pyx file has been changed but also if any dependencies within that source file have been changed (regardless of any changes to the .pyx file). These dependencies are tracked in a file with the .pyxdeps extension and the same name as the base .pyx file. The file definitions in this file can also be glob patterns.

Here's an example using C:

First we define a function in lib/examples.c

In [54]:
import os
os.makedirs('lib', exist_ok=True)

In [1]:
%%writefile lib/examples.c
#include <stdio.h>
#include <string.h>
#include "examples.h"

double fibby(const int n)
{
  int i;
  double tmp, a=0.0, b=1.0;
  for ( i = 0; i < n; i++)
  {
    tmp = a + b;
    b = a;
    a = tmp;
  }
  return a;
  // printf("Your fibonacci number is: %f\n", a);
}

Overwriting lib/examples.c


The packed C library will be compiled from the examples.h header file. In the header file we need to point to the fibby function defined in examples.c

In [2]:
%%writefile lib/examples.h
#ifndef EXAMPLES_H
#define EXAMPLES_H

double fibby(const int n);

#endif

Overwriting lib/examples.h


**Create the library**

Now we create the library by either:

```bash
$ cd lib/
$ make
```

or by executing this cell:

In [3]:
%%bash
cd lib
gcc -c -fPIC examples.c examples.h # compile the source files without making them executable
# also, compilation without -c would not work, because no main function is defined in examples.c
ar rcs libexamples.a examples.o # make an archive out of the output. This archive can be imported to cython
# ls

The finished libary is in the libexamples.a file. We need a .pyx file that makes use of our fibby function. Here it is:

In [5]:
%%writefile pyexamples.pyx
cdef extern from "examples.h":
    double fibby(const int n)

def c_wrapped_fib(n: int) -> None:
    out = fibby(n)
    print("Your fibonacci number is: ", out)

Overwriting pyexamples.pyx


And we need a setup.py file to make distutils handle the compilation. Because we have an external dependency, we need to declare that in the setup.py file.

In [6]:
%%writefile setup.py
from distutils.core import setup
from distutils.extension import Extension
from Cython.Build import cythonize

examples_extension = Extension(
    name="pyexamples",
    sources=["pyexamples.pyx"],
    libraries=["examples"],
    library_dirs=["lib"],
    include_dirs=["lib"]
)
setup(
    name="pyexamples",
    ext_modules=cythonize([examples_extension])
)

Overwriting setup.py


After executing the setup script we can use our fibby function.

In [7]:
!python setup.py build_ext --inplace

Compiling pyexamples.pyx because it changed.
[1/1] Cythonizing pyexamples.pyx
/home/kevin/.conda/envs/work_3/lib/python3.6/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/kevin/projects/cython_workshop/pyexamples.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'pyexamples' extension
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -m64 -fPIC -m64 -fPIC -fPIC -Ilib -I/home/kevin/.conda/envs/work_3/include/python3.6m -c pyexamples.c -o build/temp.linux-x86_64-3.6/pyexamples.o
gcc -pthread -shared -L/home/kevin/.conda/envs/work_3/lib -Wl,-rpath=/home/kevin/.conda/envs/work_3/lib,--no-as-needed -L/home/kevin/.conda/envs/work_3/lib -Wl,-rpath=/home/kevin/.conda/envs/work_3/lib,--no-as-needed build/temp.linux-x86_64-3.6/pyexamples.o -Llib -L/home/kevin/.conda/envs/work_3/lib -lexamples -lpython3.6m -o /home

In [1]:
import pyexamples

In [4]:
pyexamples.c_wrapped_fib(90)

('Your fibonacci number is: ', 2.880067194370816e+18)


## Python

A simple python function to print a certain number of prime numbers. I got a time of 30.7 ms ± 300 µs for for 1000 prime numbers.

In [1]:
def primes_python(nb_primes):
    if nb_primes > 1000:
        nb_primes = 1000
      
    p = list(range(100000)) # number space for checking
    len_p = 0  # The current number of elements in p.
    n = 2
    
    while len_p < nb_primes:
        # Is n prime?
        for i in p[:len_p]:
            if n % i == 0:
                break

        # If no break occurred in the loop, we have a prime.
        else:
            p[len_p] = n
            len_p += 1
        n += 1
        
    result_as_list  = [prime for prime in p[:len_p]]
    return result_as_list

In [22]:
%%timeit
primes = primes_python(1000)

30.7 ms ± 300 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Cython

By simply saving the function to a .pyx file and compile it with cython we can save a great amount of time.

In [23]:
%%writefile prime_numbers.pyx
def primes_cython(nb_primes):
    if nb_primes > 1000:
        nb_primes = 1000
      
    p = list(range(100000)) # number space for checking
    len_p = 0  # The current number of elements in p.
    n = 2
    
    while len_p < nb_primes:
        # Is n prime?
        for i in p[:len_p]:
            if n % i == 0:
                break

        # If no break occurred in the loop, we have a prime.
        else:
            p[len_p] = n
            len_p += 1
        n += 1
        
    result_as_list  = [prime for prime in p[:len_p]]
    return result_as_list

Overwriting prime_numbers.pyx


In [24]:
%%writefile setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("prime_numbers.pyx")
)

Overwriting setup.py


In [25]:
!python setup.py build_ext --inplace

Compiling prime_numbers.pyx because it changed.
[1/1] Cythonizing prime_numbers.pyx
/home/kevin/.conda/envs/work_3/lib/python3.6/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/kevin/projects/accelerate_python_code/prime_numbers.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'prime_numbers' extension
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -m64 -fPIC -m64 -fPIC -fPIC -I/home/kevin/.conda/envs/work_3/include/python3.6m -c prime_numbers.c -o build/temp.linux-x86_64-3.6/prime_numbers.o
gcc -pthread -shared -L/home/kevin/.conda/envs/work_3/lib -Wl,-rpath=/home/kevin/.conda/envs/work_3/lib,--no-as-needed -L/home/kevin/.conda/envs/work_3/lib -Wl,-rpath=/home/kevin/.conda/envs/work_3/lib,--no-as-needed build/temp.linux-x86_64-3.6/prime_numbers.o -L/home/kevin/.conda/envs/work_3/lib -lpython3.6m -o 

In [28]:
import os
os.listdir()

['.ipynb_checkpoints',
 'vmdscene.obj',
 'vmdscene.mtl',
 'accelerate_python_code.ipynb',
 'accelerate_python_code.pdf',
 'helloworld.pyx',
 'helloworld.c',
 'helloworld.cpython-36m-x86_64-linux-gnu.so',
 'setup.py',
 'build',
 'prime_numbers.pyx',
 'prime_numbers_cdef.pyx',
 'prime_numbers_cdef.c',
 'prime_numbers_cdef.cpython-36m-x86_64-linux-gnu.so',
 'prime_numbers.c',
 '.nfs000000000b7a973100015283',
 'prime_numbers.cpython-36m-x86_64-linux-gnu.so']

**Importing from shared objects is 

In [6]:
from prime_numbers import primes_cython

In [7]:
%%time
primes = primes_cython(1000)
print(primes[::4])

[2, 11, 23, 41, 59, 73, 97, 109, 137, 157, 179, 197, 227, 241, 269, 283, 313, 347, 367, 389, 419, 439, 461, 487, 509, 547, 571, 599, 617, 643, 661, 691, 727, 751, 773, 811, 829, 859, 883, 919, 947, 977, 1009, 1031, 1051, 1087, 1103, 1129, 1171, 1201, 1229, 1259, 1289, 1303, 1327, 1381, 1427, 1447, 1471, 1489, 1523, 1553, 1579, 1607, 1621, 1663, 1697, 1723, 1753, 1787, 1823, 1867, 1879, 1913, 1951, 1993, 2011, 2039, 2081, 2099, 2131, 2153, 2207, 2239, 2269, 2293, 2333, 2351, 2381, 2399, 2437, 2467, 2521, 2549, 2591, 2621, 2659, 2683, 2699, 2719, 2749, 2789, 2803, 2843, 2879, 2909, 2953, 2971, 3019, 3049, 3083, 3121, 3169, 3203, 3229, 3259, 3307, 3329, 3359, 3389, 3433, 3463, 3499, 3529, 3547, 3581, 3613, 3637, 3673, 3701, 3733, 3769, 3803, 3847, 3877, 3911, 3929, 3967, 4007, 4027, 4073, 4099, 4133, 4159, 4217, 4241, 4261, 4289, 4339, 4373, 4421, 4451, 4483, 4517, 4549, 4591, 4637, 4651, 4679, 4723, 4759, 4793, 4817, 4877, 4919, 4943, 4969, 4999, 5021, 5059, 5099, 5119, 5171, 5209, 5237,

## Cython with C-types

In [8]:
%%writefile prime_numbers_cdef.pyx
def primes_cython_cdef(int nb_primes):
    cdef int n, i, len_p
    cdef int p[1000]
    if nb_primes > 1000:
        nb_primes = 1000

    len_p = 0  # The current number of elements in p.
    n = 2
    while len_p < nb_primes:
        # Is n prime?
        for i in p[:len_p]:
            if n % i == 0:
                break

        # If no break occurred in the loop, we have a prime.
        else:
            p[len_p] = n
            len_p += 1
        n += 1

    # Let's return the result in a python list:
    result_as_list  = [prime for prime in p[:len_p]]
    return result_as_list

Writing prime_numbers_cdef.pyx


In [9]:
%%writefile setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("prime_numbers_cdef.pyx")
)

Overwriting setup.py


In [10]:
!python setup.py build_ext --inplace

Compiling prime_numbers_cdef.pyx because it changed.
[1/1] Cythonizing prime_numbers_cdef.pyx
/home/kevin/.conda/envs/work_3/lib/python3.6/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/kevin/projects/accelerate_python_code/prime_numbers_cdef.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'prime_numbers_cdef' extension
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -m64 -fPIC -m64 -fPIC -fPIC -I/home/kevin/.conda/envs/work_3/include/python3.6m -c prime_numbers_cdef.c -o build/temp.linux-x86_64-3.6/prime_numbers_cdef.o
gcc -pthread -shared -L/home/kevin/.conda/envs/work_3/lib -Wl,-rpath=/home/kevin/.conda/envs/work_3/lib,--no-as-needed -L/home/kevin/.conda/envs/work_3/lib -Wl,-rpath=/home/kevin/.conda/envs/work_3/lib,--no-as-needed build/temp.linux-x86_64-3.6/prime_numbers_cdef.o -L/home/kevin/.con

In [14]:
from prime_numbers_cdef import primes_cython_cdef

In [16]:
%%time
primes = primes_cython_cdef(1000)
print(primes[::4])

[2, 11, 23, 41, 59, 73, 97, 109, 137, 157, 179, 197, 227, 241, 269, 283, 313, 347, 367, 389, 419, 439, 461, 487, 509, 547, 571, 599, 617, 643, 661, 691, 727, 751, 773, 811, 829, 859, 883, 919, 947, 977, 1009, 1031, 1051, 1087, 1103, 1129, 1171, 1201, 1229, 1259, 1289, 1303, 1327, 1381, 1427, 1447, 1471, 1489, 1523, 1553, 1579, 1607, 1621, 1663, 1697, 1723, 1753, 1787, 1823, 1867, 1879, 1913, 1951, 1993, 2011, 2039, 2081, 2099, 2131, 2153, 2207, 2239, 2269, 2293, 2333, 2351, 2381, 2399, 2437, 2467, 2521, 2549, 2591, 2621, 2659, 2683, 2699, 2719, 2749, 2789, 2803, 2843, 2879, 2909, 2953, 2971, 3019, 3049, 3083, 3121, 3169, 3203, 3229, 3259, 3307, 3329, 3359, 3389, 3433, 3463, 3499, 3529, 3547, 3581, 3613, 3637, 3673, 3701, 3733, 3769, 3803, 3847, 3877, 3911, 3929, 3967, 4007, 4027, 4073, 4099, 4133, 4159, 4217, 4241, 4261, 4289, 4339, 4373, 4421, 4451, 4483, 4517, 4549, 4591, 4637, 4651, 4679, 4723, 4759, 4793, 4817, 4877, 4919, 4943, 4969, 4999, 5021, 5059, 5099, 5119, 5171, 5209, 5237,

# Reloading cython

Cython is a little bit complicated when it comes to reloading the compiled .so files.

# Import C and C++ code (chapter 7 and 8 from book)

# Convert to pdf

To build this pdf you need to have jupyter extensions installed. Either with pip:

```bash
$ pip install jupyter_contrib_nbextensions
```

or with conda:

```bash
$ conda install -c conda-forge jupyter_contrib_nbextensions
```

To convert the notebook into a pdf you can execute:

```bash
$ jupyter nbconvert --to pdf accelerate_python_code.ipynb
```

### Errors
These errors might occur when you try to convert this notebook into a pdf:

**Permissions to a shared object**

If you get an error, because python does not have permissions to some shared object execute this:

```bash
$ export LD_LIBRARY_PATH=/home/kevin/.conda/envs/work_3/lib:$LD_LIBRARY_PATH
```
**Wrong kernelspec**

If you get an error, because conda doesn't know the kernel open the notebook file in a texteditor and change the kernel name by hand.

```json
 "metadata": {
  "kernelspec": {
   "display_name": "work_3",
   "language": "python",
   "name": "nb-conda-faster_py_env-py"
```

will be changed to:

```json
 "metadata": {
  "kernelspec": {
   "display_name": "work_3",
   "language": "python",
   "name": "faster_py_env"
```

In [3]:
!jupyter nbconvert --to pdf accelerate_python_code.ipynb

[NbConvertApp] Converting notebook accelerate_python_code.ipynb to pdf
[NbConvertApp] Writing 24118 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 32661 bytes to accelerate_python_code.pdf
